# Reaction-level CAL Theorem Examples

In [1]:
// Max-plus functions from https://github.com/msnhdyt/Max-Plus-Algebra
// Type definitions
type Matrix = number[][];
type Vector = number[];

// Constants
const EPS = Number.NEGATIVE_INFINITY; // epsilon = empty element = -inf
const INF = Number.POSITIVE_INFINITY; // positive infinity

// Function for calculating A oplus B
function oplus(A: Matrix, B: Matrix): Matrix {
  if (A.length !== B.length || A[0].length !== B[0].length) {
    throw new TypeError(`A and B must have the same shape: A.shape=[${A.length},${A[0].length}], B.shape=[${B.length},${B[0].length}]`);
  }
  
  const result: Matrix = [];
  for (let i = 0; i < A.length; i++) {
    result[i] = [];
    for (let j = 0; j < A[i].length; j++) {
      result[i][j] = Math.max(A[i][j], B[i][j]);
    }
  }
  return result;
}

// Function for calculating A otimes B, i.e., matrix multiplication
function otimes(A: Matrix, B: Matrix): Matrix {
  // console.log("A dim:");
  // console.log(A);
  // console.log("B dim:");
  // console.log(B);
  if (A[0].length !== B.length) {
    throw new TypeError(`A's 2nd dimension does not match B's 1st dimension: A.shape=[${A.length},${A[0].length}], B.shape=[${B.length},${B[0].length}]`);
  }

  if (A.length === 1 && A[0].length === 1 || B.length === 1 && B[0].length === 1) {
    return [[A[0][0] + B[0][0]]];
  } else {
    const result: Matrix = [];
    const B_T = transpose(B);
    
    for (let i = 0; i < A.length; i++) {
      result[i] = [];
      for (let j = 0; j < B_T.length; j++) {
        result[i][j] = odot(A[i], B_T[j]);
      }
    }
    return result;
  }
}

// Function for taking the max-plus dot product of two row vectors
function odot(a: Vector, b: Vector): number {
  if (a.length !== b.length) {
    throw new Error(`Vector a and b must have the same length: a.length=${a.length}, b.length=${b.length}`);
  }
  
  const result: number[] = [];
  for (let i = 0; i < a.length; i++) {
    result[i] = a[i] + b[i];
  }
  return Math.max(...result);
}

// Helper function to transpose a matrix
function transpose(matrix: Matrix): Matrix {
  return matrix[0].map((_, colIndex) => matrix.map(row => row[colIndex]));
}

// Function for calculating A^n in otimes
function powOtimes(A: Matrix, n: number): Matrix {
  let result = A;
  for (let i = 0; i < n - 1; i++) {
    result = otimes(result, A);
  }
  return result;
}

// Function for calculating trace(A)
function trace(A: Matrix): number {
  const diagonal: number[] = [];
  for (let i = 0; i < A.length; i++) {
    diagonal.push(A[i][i]);
  }
  return Math.max(...diagonal);
}

// Function for calculating A*
function star(A: Matrix): Matrix {
  const n = A.length;
  // Initialize result with E (identity matrix)
  let result: Matrix = [];
  for (let i = 0; i < n; i++) {
    result[i] = [];
    for (let j = 0; j < n; j++) {
      if (i === j) {
        result[i][j] = 0;
      } else {
        result[i][j] = EPS;
      }
    }
  }

  for (let i = 1; i < n; i++) {
    result = oplus(result, powOtimes(A, i));
  }
  
  return result;
}

// Function for calculating A^+
function plus(A: Matrix): Matrix {
  let result = A;
  const n = A.length;
  
  for (let i = 1; i <= n; i++) {
    result = oplus(result, powOtimes(A, i));
  }
  
  return result;
}

console.log("Max-plus algebra functions loaded");


Max-plus algebra functions loaded


In [2]:
// Example matrix A
const A: Matrix = [
  [-2, 3, 1],
  [1, 1, EPS],
  [EPS, 2, 1]
];

console.log("Matrix A:", A);
console.log("Trace of A:", trace(A));


Matrix A: [ [ -2, 3, 1 ], [ 1, 1, -Infinity ], [ -Infinity, 2, 1 ] ]
Trace of A: 1


### Impossible Consistency

<img src="../../img/ImpossibleConsistency.svg" alt="drawing" width="500"/>

In the formulation below, reaction 1s are ignored for simplicity.


### Step 0: The cell below sets all parameters.


In [3]:
// System setup and evolution equations
// Return an epsilon-filled matrix with the same shape as the input matrix
function reset(a: Matrix): Matrix {
  return a.map(row => row.map(() => EPS));
}

// Create matrices. These are row matrices, i.e., shape = 1 x n
let x_p   : Matrix = [[EPS, EPS, EPS, EPS]]; // Physical firing times in the k+1-th iteration
let x     : Matrix = [[EPS, EPS, EPS, EPS]]; // Physical firing times in the k-th iteration
let t     : number = EPS;
let t_bar : Matrix = [new Array(4).fill(t)]; // Logical scheduling times of physical actions

// Indices
const a1 = 0; // Reaction 1 in reactor A
const a2 = 1; // Reaction 2 in reactor A
const b1 = 2; // Reaction 1 in reactor B
const b2 = 3; // Reaction 2 in reactor B
const pa = 0; // Physical action in reactor A
const pb = 1; // Physical action in reactor B

// Setting execution times
const e: Matrix = [[1, 1, 1, 1]];   // Execution times

// Initial firing of actions
const paInitOffset = 0;
const pbInitOffset = 0;

// Set the period of the actions 
// (Either same or one is -inf. See NOTE above)
// NOTE 2: eigenvalue(M) = 2. Ie., smallest minimum spacing of the physical action to make the system feasible
const paPeriod = 2; // If actions scheduled simultaneously, use 1.999 to see divergence. Use inf if no future events are scheduled.
const pbPeriod = 2; // If actions scheduled simultaneously, use 1.999 to see divergence. Use inf if no future events are scheduled.

//// Not tested well
// This is used to model the case where the physical actions
// are scheduled with period > 2 but slightly misaligned.
// In this case, divergence still occurs.
// To see this effect, set paInitOffset=0 and pbInitOffset=0.001 (small misalignment),
// then set paPeriod=pbPeriod=3.999 to observe
// divergence, which disappears when paPeriod=pbPeriod=4
// let toggle = true;
// let paCache = u[0][pa];
// let pbCache = u[0][pb];

// STAs 
/*
const staA = 1;
const staB = 1;
*/

// Iteration count
let k = 0;

// Reset x to eps
x = reset(x);

console.log("Parameters set up");


Parameters set up


In [ ]:
// Reaction dependency matrix
const GammaStar: Matrix = [
  [0, EPS, EPS, EPS],
  [e[0][a1], 0, e[0][b1], EPS],
  [EPS, EPS, 0, EPS],
  [e[0][a1], EPS, e[0][b1], 0],
];

// Barrier synchronization matrix
const B: Matrix = [
  [e[0][a1], e[0][a2], EPS, EPS],
  [e[0][a1], e[0][a2], EPS, EPS],
  [EPS, EPS, e[0][b1], e[0][b2]],
  [EPS, EPS, e[0][b1], e[0][b2]],
];

In [5]:
// Step function for evolution (updated to work with matrices)
function step(x: Matrix, t: Matrix): Matrix {
  // Transpose input matrices for matrix multiplication
  const xT: Matrix = transpose(x);
  const tT: Matrix = transpose(t);

  const GammaStarB = otimes(GammaStar, B);
  const GammaStarBxT = otimes(GammaStarB, xT);
  const GammaStartT = otimes(GammaStar, tT);
  
  const result = oplus(GammaStarBxT, GammaStartT);

  // Return the transpose of the result
  return transpose(result);
}


In [6]:
// Reporting function (updated to work with matrices)
function reportStats(k: number, x: Matrix, t: number): void {
  const lagK = x[0].map(xi => xi - t); // lag

  console.log(`Tag index k=${k}`);
  console.log(`t(k)= ${t}`);
  console.log(`x(k) = earliest possible firing times = [${x[0].join(', ')}]`);
  console.log(`lag(k) = [${lagK.join(', ')}]`);
}  

### Step 1: Execute the following cell *once*

In [7]:
let paNextTime = paInitOffset;
let pbNextTime = pbInitOffset;
t = Math.min(paNextTime, pbNextTime);
t_bar = [new Array(4).fill(t)];

x = step(x, t_bar);
k += 1;

// Statistics
reportStats(k, x, t);


Tag index k=1
t(k)= 0
x(k) = earliest possible firing times = [0, 1, 0, 1]
lag(k) = [0, 1, 0, 1]


### Last Step: Execute the following cell *repeatedly*


In [17]:
// Bump the next physical action firing times
paNextTime += paPeriod;
pbNextTime += pbPeriod;
t = Math.min(paNextTime, pbNextTime);
t_bar = [new Array(4).fill(t)];

//// Not tested well
// if (toggle) {
//     u[0][pa] = paCache + paPeriod;
//     pbCache = u[0][pb];
//     u[0][pb] = EPS;
// } else {
//     paCache = u[0][pa];
//     u[0][pa] = EPS;
//     u[0][pb] = pbCache + pbPeriod;
// }
// toggle = !toggle;
// console.log(`toggle=${toggle}`);

// Step the system from k to k+1
x = step(x, t_bar);
k += 1;

// Statistics
reportStats(k, x, t);


Tag index k=11
t(k)= 20
x(k) = earliest possible firing times = [20, 21, 20, 21]
lag(k) = [0, 1, 0, 1]
